In [1]:
from collections import defaultdict
from tqdm import tqdm
import copy
from nltk.tokenize import word_tokenize
import math

In [2]:
import json
# Specify the path to your JSON file
file_path = '/kaggle/input/finaldataset/FinalDataset (2).json'
# Open the file and read its content
with open(file_path, 'r') as file:
    data = json.load(file)

In [3]:
import json
# Specify the path to your JSON file
file_path = "/kaggle/input/finaldataset2/mini_wiki_collection (1).json"
# Open the file and read its content
with open(file_path, 'r') as file:
    data2 = json.load(file)

In [8]:
documents = []
for i in range(len(data2)):
    temp = ""
    for j in range(len(data2[i]["text"])):
        temp = temp + data2[i]["text"][j]
    documents.append(temp)

In [9]:
values = data.values()
queries = []
for value in values:
    queries.append(value['input'])

In [10]:
remove = ['.', ',', '!', '?', '[', ']', '{', '}', '(', ')', '*', '&', '#', ':', ';', '"', "'", '-', '+', '_', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
remove2= ['\n', '”', '“', '’']

In [11]:
documents2 = []
for document in tqdm(documents):
    document2 = copy.deepcopy(document)
    for c in remove:
        document2 = document2.replace(c, '')
    for c in remove2:
        document2 = document2.replace(c, ' ')
    documents2.append(document2)
documents = copy.deepcopy(documents2)

100%|██████████| 1000/1000 [00:00<00:00, 2552.33it/s]


In [12]:
queries2 = []
for query in tqdm(queries):
    query2 = copy.deepcopy(query)
    for c in remove:
        query2 = query2.replace(c, '')
    for c in remove2:
        query2 = query2.replace(c, ' ')
    queries2.append(query2)
queries = copy.deepcopy(queries2)

100%|██████████| 1000/1000 [00:00<00:00, 92237.24it/s]


In [13]:
documents_tokenized = []
for document in tqdm(documents):
    # words = documents[document].split()
    words = word_tokenize(document)
    words2 = []
    for word in words:
        word = word.lower()
        words2.append(word)
    documents_tokenized.append(words2)

100%|██████████| 1000/1000 [00:11<00:00, 88.64it/s]


In [14]:
queries_tokenized = []
for query in tqdm(queries):
    # words = queries[query].split()
    words = word_tokenize(query)
    words2 = []
    for word in words:
        word = word.lower()
        words2.append(word)
    queries_tokenized.append(words2)

100%|██████████| 1000/1000 [00:00<00:00, 6182.30it/s]


In [17]:
tf = defaultdict(lambda: {})
freq = defaultdict(lambda: 0)
for i in tqdm(range(len(documents_tokenized))):
    tf[i] = defaultdict(lambda: 0)
    tokens = documents_tokenized[i]
    for token in tokens:
        freq[token] += 1
        tf[i][token] += 1

for i in tqdm(range(len(documents_tokenized))):
    for token in tf[i].keys():
        tf[i][token] = tf[i][token] / len(documents_tokenized[i])

100%|██████████| 1000/1000 [00:00<00:00, 2501.14it/s]


In [18]:
idf = defaultdict(lambda: 0)
ndoc = defaultdict(lambda: 0)
for i in tqdm(range(len(documents_tokenized))):
    temp = defaultdict(lambda: 0)
    tokens = documents_tokenized[i]
    for token in tokens:
        if(temp[token] == 0):
            idf[token] += 1
            temp[token] += 1

for token in tqdm(idf.keys()):
    ndoc[token] = idf[token]
    idf[token] = math.log(len(documents_tokenized) / idf[token])

100%|██████████| 100363/100363 [00:00<00:00, 613956.20it/s]


In [19]:
tf_idf = defaultdict(lambda: 0)
for i in tqdm(range(len(documents_tokenized))):
    tf_idf[i] = defaultdict(lambda: 0)
    for token in documents_tokenized[i]:
        tf_idf[i][token] = tf[i][token] * idf[token]

100%|██████████| 1000/1000 [00:01<00:00, 668.25it/s]


In [21]:
sum = 0
for document in documents_tokenized:
    sum += len(document)
avgdl = sum / len(documents_tokenized)

In [32]:
k1 = 1.75
b = 0.75

In [29]:
def IDF(token):
    k = math.log(((len(documents_tokenized) - ndoc[token] + 0.5) / (ndoc[token] + 0.5)) + 1)
    return k

In [30]:
def bm25(query, document_number):
    sum = 0
    for token in query:
        f = tf[document_number][token]
        idf_query = IDF(token)
        sum = sum + ((idf_query * f * (k1 + 1)) / (f + (k1 * (1 - b + (b * (len(documents_tokenized[document_number]) / avgdl))))))
    return sum

In [33]:
bm25_dict = defaultdict(lambda: 0)
for i in tqdm(range(len(queries_tokenized))):
    for j in range(len(documents_tokenized)):
        bm25_dict[(i, j)] = bm25(queries_tokenized[i], j)

100%|██████████| 1000/1000 [00:24<00:00, 40.91it/s]
